<a href="https://colab.research.google.com/github/djdumpling/Machine-Learning-Prince-/blob/main/7_2_Backpropagation_(needs_fixing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)

# Number of layers
K=5
# Number of neurons per layer
D=6
# Input layer
D_i = 1
# Output layer
D_o = 1

# Make empty lists
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Create input and output layers
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size =(D,1))
all_biases[-1]= np.random.normal(size =(D_o,1))

# Create intermediate layers
for k in range(1, K):
    all_weights[k] = np.random.normal(size=(D, D))
    all_biases[k] = np.random.normal(size=(D, 1))

In [ ]:
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

In [ ]:
def compute_network_output(net_input, all_weights, all_biases):
  K = len(all_weights) - 1

  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  all_h[0] = net_input

  for layer in range(K):
    all_f[layer] = np.matmul(all_weights[layer], all_h[layer]) + all_biases[layer]
    all_h[layer+1] = ReLU(all_f[layer])

  all_f[K] = np.matmul(all_weights[K], all_h[K]) + all_biases[K]

  net_output = all_f[K]

  return net_output, all_f, all_h

In [ ]:
net_input = np.ones((D_i,1)) * 1.2
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 1.907


In [ ]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
  return 2*(net_output-y)

y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 327.371


<ipython-input-6-b8e329e9b0a5>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("y = %3.3f Loss = %3.3f"%(y, loss))


In [ ]:
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

In [ ]:
def backward_pass(all_weights, all_biases, all_f, all_h, y):

  K = len(all_weights) - 1

  # store derivative dl_dweights and dl_dbiases in lists as well
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)

  # store derivatives of the loss w.r.t activation and preactivation in lists
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)

  #sticking with the convention that all_h[0] is the net input and all_f[-1] is the net output

  all_dl_df[K] = np.array(d_loss_d_output(all_f[K], y))

  for layer in range(K, -1, -1):
    # TODO Calculate the derivatives of the loss with respect to the biases at layer from all_dl_df[layer]. (eq 7.21)
    # NOTE!  To take a copy of matrix X, use Z=np.array(X)
    all_dl_dbiases[layer] = np.array(all_dl_df[layer])

    # TODO Calculate the derivatives of the loss with respect to the weights at layer from all_dl_df[layer] and all_h[layer] (eq 7.22)
    all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].transpose())

    # TODO: calculate the derivatives of the loss with respect to the activations from weight and derivatives of next preactivations (second part of last line of eq 7.24)
    all_dl_dh[layer] = np.matmul(all_weights[layer].transpose(), all_dl_df[layer])

    if layer > 0:
      # TODO Calculate the derivatives of the loss with respect to the pre-activation f (use derivative of ReLu function, first part of last line of eq. 7.24)
      all_dl_df[layer-1] = indicator_function(all_f[layer]) * all_dl_dh[layer]

  return all_dl_dweights, all_dl_dbiases

In [ ]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

In [ ]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# Test the dervatives of the bias vectors
for layer in range(K):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])
  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

# Test the derivatives of the weights matrices
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[ -0.   ]
 [ 24.469]
 [  0.   ]
 [  0.   ]
 [-30.963]
 [ -0.   ]]
Bias 0, derivatives from finite differences
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[  0.   ]
 [  0.   ]
 [-44.856]
 [ -0.   ]
 [ -2.557]
 [ 18.601]]
Bias 1, derivatives from finite differences
[[  0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[  0.   ]
 [ 30.165]
 [ 18.282]
 [ -0.   ]
 [ 24.196]
 [-30.257]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Failure!  Derivatives different.
-----------------------------------------------
Bias 3, derivatives from backprop:
[[-0.   ]
 [ 0.   ]
 [-0.   ]
 [ 0.   ]
 